# DDI InfluxDB Timeseries

### Imports

In [ ]:
# Imports
import os 
import pandas as pd
from datetime import datetime, timezone
from dotenv import load_dotenv
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from libs.fetch_data import fetch_data

### Load of env Variables

In [ ]:
# Constants
load_dotenv(dotenv_path=".env")
URL   = os.getenv("INFLUX_URL")
ORG   = os.getenv("INFLUX_ORG", "basel-lab")
TOKEN = os.getenv("INFLUX_TOKEN")
BUCKET = os.getenv("BUCKET_PARKING_RAW", "basel_parking_raw")
PARKING_URL = os.getenv("PARKING_URL")
RAIN_URL = os.getenv("RAIN_URL")
ALRDY_INSTALLED = os.getenv("ALRDY_INSTALLED", 'False').lower() in ('true', '1', 't')  

### Fetch of Data

In [ ]:
if not ALRDY_INSTALLED:
    df_parking, df_rain = fetch_data(PARKING_URL, RAIN_URL)
    df_parking.to_csv("../data/parking.csv")
    df_rain.to_csv("../data/rain.csv")

### Data analysis for import in InfluxDB

In [ ]:

df_parking = pd.read_csv("../data/parking.csv", sep=",")
df_rain = pd.read_csv("../data/rain.csv", sep=",")

ts = pd.to_datetime(df_parking['published'], errors="coerce", utc=True)
df_parking['time'] = ts


df_rain['date'] = pd.to_datetime(df_rain['date'], errors="coerce")
df_rain['time'] = (
    df_rain['date']
    + pd.to_timedelta(12, unit="h")
).dt.tz_localize("Europe/Zurich", nonexistent="shift_forward", ambiguous="NaT"
).dt.tz_convert("UTC")


df_rain.head(10)

### Make Connection

In [ ]:
# Connection 
assert TOKEN, "Bitte Token in .env file hinzufügen"
client = InfluxDBClient(url=URL, token=TOKEN, org=ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()
health = client.health()
print("InfluxDB health:", health.status, "-", health.message)
